In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [6]:
subject = 'Rio Grande do Sul - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [7]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_RS.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Rio Grande do Sul - IDH Longevidade,Rio Grande do Sul - IDH,Rio Grande do Sul - IDH Renda,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande do Sul - IDH Educacao,Rio Grande do Sul - Desemprego,...,Taxa Selic (%),Rio Grande Do Sul - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio Grande do Sul - Consumo de Cimento (t)
0,2008,0.847908,0.778585,0.783917,2.188016e+08,9.232116e+06,18.525103,1.994837e+08,0.710282,7.956795,...,11.760983,1892.120992,21.764081,1009.667623,1159.549843,19.539269,14.577965,75.001024,1.589194,2718.542000
1,2009,0.848897,0.779263,0.782790,2.203195e+08,9.292192e+06,18.558136,1.997999e+08,0.712521,7.985037,...,9.472585,1987.964633,22.056812,1009.481089,1148.622979,19.521903,14.707061,75.640304,1.544445,2685.740000
2,2010,0.849366,0.779976,0.782082,2.258308e+08,9.359876e+06,18.639011,2.009995e+08,0.714897,7.949439,...,9.332485,2093.049582,21.905564,1009.521141,1180.776262,19.504051,14.610253,75.347633,1.511781,3070.312000
3,2011,0.849325,0.780726,0.781784,2.307298e+08,9.370894e+06,18.687305,2.017727e+08,0.717405,7.936698,...,10.985689,2163.054486,22.168556,1009.586912,1208.677123,19.639685,14.612190,74.832405,1.485231,3399.177000
4,2012,0.848674,0.781171,0.781580,2.352214e+08,9.350813e+06,18.711648,2.024297e+08,0.719559,6.613682,...,8.159014,2145.073736,22.605974,1009.662812,1214.063067,19.834806,14.847869,75.069595,1.460267,3487.789000
5,2013,0.847730,0.782685,0.782795,2.392894e+08,9.261936e+06,18.735087,2.030530e+08,0.723355,7.062492,...,7.913561,2014.296886,23.627714,1009.741901,1201.461810,20.057230,15.300827,76.128780,1.411746,3505.720000
6,2014,0.846291,0.783492,0.783612,2.421797e+08,9.200346e+06,18.727522,2.030899e+08,0.726051,7.423718,...,10.350442,1855.561669,24.400941,1009.762470,1171.719265,20.146399,15.684750,77.480667,1.400940,3226.786714
7,2015,0.844315,0.783867,0.784163,2.442913e+08,9.140544e+06,18.713368,2.030056e+08,0.728306,7.683986,...,12.454398,1702.806499,24.916446,1009.840981,1172.307462,20.091834,15.811951,78.312245,1.426194,2978.617371
8,2016,0.850637,0.784838,0.776923,2.455025e+08,9.097791e+06,18.694019,2.028122e+08,0.732388,7.674279,...,13.111908,1556.399041,25.386577,1009.893214,1192.750068,20.140420,15.954377,78.787281,1.473465,2744.168914
9,2017,0.847968,0.784459,0.777370,2.457104e+08,9.054261e+06,18.678679,2.026081e+08,0.733333,7.404242,...,9.479812,1456.194220,24.738710,1009.894951,1193.749290,20.268929,16.171905,79.268644,1.530121,2703.746000


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Rio Grande do Sul - IDH Longevidade,Rio Grande do Sul - IDH,Rio Grande do Sul - IDH Renda,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande do Sul - IDH Educacao,Rio Grande do Sul - Desemprego,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rio Grande Do Sul - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,0.847908,0.778585,0.783917,2.188016e+08,9.232116e+06,18.525103,1.994837e+08,0.710282,7.956795,5.483335e+08,...,2.868108,11.760983,1892.120992,21.764081,1009.667623,1159.549843,19.539269,14.577965,75.001024,1.589194
1,0.848897,0.779263,0.782790,2.203195e+08,9.292192e+06,18.558136,1.997999e+08,0.712521,7.985037,5.302764e+08,...,2.305885,9.472585,1987.964633,22.056812,1009.481089,1148.622979,19.521903,14.707061,75.640304,1.544445
2,0.849366,0.779976,0.782082,2.258308e+08,9.359876e+06,18.639011,2.009995e+08,0.714897,7.949439,5.046425e+08,...,1.958246,9.332485,2093.049582,21.905564,1009.521141,1180.776262,19.504051,14.610253,75.347633,1.511781
3,0.849325,0.780726,0.781784,2.307298e+08,9.370894e+06,18.687305,2.017727e+08,0.717405,7.936698,4.854622e+08,...,2.079088,10.985689,2163.054486,22.168556,1009.586912,1208.677123,19.639685,14.612190,74.832405,1.485231
4,0.848674,0.781171,0.781580,2.352214e+08,9.350813e+06,18.711648,2.024297e+08,0.719559,6.613682,4.867661e+08,...,2.183282,8.159014,2145.073736,22.605974,1009.662812,1214.063067,19.834806,14.847869,75.069595,1.460267
5,0.847730,0.782685,0.782795,2.392894e+08,9.261936e+06,18.735087,2.030530e+08,0.723355,7.062492,5.225848e+08,...,2.403445,7.913561,2014.296886,23.627714,1009.741901,1201.461810,20.057230,15.300827,76.128780,1.411746
6,0.846291,0.783492,0.783612,2.421797e+08,9.200346e+06,18.727522,2.030899e+08,0.726051,7.423718,5.676413e+08,...,2.748397,10.350442,1855.561669,24.400941,1009.762470,1171.719265,20.146399,15.684750,77.480667,1.400940
7,0.844315,0.783867,0.784163,2.442913e+08,9.140544e+06,18.713368,2.030056e+08,0.728306,7.683986,5.179392e+08,...,2.501529,12.454398,1702.806499,24.916446,1009.840981,1172.307462,20.091834,15.811951,78.312245,1.426194
8,0.850637,0.784838,0.776923,2.455025e+08,9.097791e+06,18.694019,2.028122e+08,0.732388,7.674279,4.524144e+08,...,2.073464,13.111908,1556.399041,25.386577,1009.893214,1192.750068,20.140420,15.954377,78.787281,1.473465
9,0.847968,0.784459,0.777370,2.457104e+08,9.054261e+06,18.678679,2.026081e+08,0.733333,7.404242,4.213689e+08,...,2.177386,9.479812,1456.194220,24.738710,1009.894951,1193.749290,20.268929,16.171905,79.268644,1.530121


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     2685.740000
1     3070.312000
2     3399.177000
3     3487.789000
4     3505.720000
5     3226.786714
6     2978.617371
7     2744.168914
8     2703.746000
9     2665.575000
10    2753.564000
11    2993.250000
12    3222.295000
13    3135.029000
14            NaN
Name: Rio Grande do Sul - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Rio Grande do Sul - IDH Longevidade,Rio Grande do Sul - IDH,Rio Grande do Sul - IDH Renda,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande do Sul - IDH Educacao,Rio Grande do Sul - Desemprego,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rio Grande Do Sul - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,0.262071,-1.744132,1.077919,-1.825005,0.212899,-2.141045,-2.061029,-1.618062,1.067231,1.204752,...,2.026352,0.984445,0.296064,-1.327319,-0.466803,-1.324966,-1.343190,-1.253487,-1.103510,1.757329
1,0.634805,-1.407889,0.658847,-1.667118,0.728720,-1.625209,-1.794456,-1.350203,1.137132,0.820657,...,0.085558,-0.051710,0.650867,-1.088422,-1.758195,-1.791087,-1.401878,-1.030567,-0.687296,0.956056
2,0.811625,-1.054359,0.395298,-1.093856,1.309868,-0.362267,-0.783129,-1.066111,1.049023,0.275392,...,-1.114490,-0.115145,1.039880,-1.211856,-1.480917,-0.419484,-1.462208,-1.197732,-0.877844,0.371177
3,0.796260,-0.682451,0.284565,-0.584284,1.404462,0.391879,-0.131266,-0.766117,1.017489,-0.132596,...,-0.697344,0.633403,1.299030,-0.997228,-1.025578,0.770718,-1.003846,-1.194387,-1.213292,-0.104225
4,0.550707,-0.461440,0.208652,-0.117084,1.232045,0.772027,0.422591,-0.508515,-2.257055,-0.104860,...,-0.337667,-0.646476,1.232468,-0.640252,-0.500116,1.000474,-0.344453,-0.787426,-1.058866,-0.551216
5,0.194856,0.289613,0.660649,0.306044,0.468938,1.138050,0.948126,-0.054430,-1.146224,0.657046,...,0.422337,-0.757614,0.748346,0.193588,0.047426,0.462926,0.407206,-0.005274,-0.369265,-1.420027
6,-0.347745,0.689931,0.964309,0.606687,-0.059888,1.019908,0.979199,0.268055,-0.252167,1.615453,...,1.613108,0.345772,0.160725,0.824617,0.189825,-0.805840,0.708545,0.657672,0.510905,-1.613509
7,-1.092658,0.875615,1.169272,0.826322,-0.573354,0.798884,0.908132,0.537677,0.392013,0.558230,...,0.760923,1.298413,-0.404758,1.245319,0.733369,-0.780748,0.524147,0.877318,1.052319,-1.161326
8,1.290931,1.357194,-1.524000,0.952305,-0.940438,0.496730,0.745088,1.025970,0.367987,-0.835563,...,-0.716758,1.596124,-0.946742,1.628992,1.094983,0.091298,0.688338,1.123254,1.361599,-0.314904
9,0.284516,1.169522,-1.357520,0.973929,-1.314196,0.257177,0.573033,1.138923,-0.300372,-1.495939,...,-0.358019,-0.048438,-1.317690,1.100269,1.107004,0.133923,1.122622,1.498874,1.675000,0.699581


In [11]:
reshaped_train_input = dfToInputRNN(train_input)

In [12]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[-1.42539061,  0.23975554, -1.08489418,  0.70308191,
         -1.05304162, -0.58353775, -0.09523565,  1.08391904,
         -0.51744599, -1.0453709 ,  1.21859064, -0.99976018,
          0.79428371,  1.15575749, -0.52559858,  1.30260559,
         -0.55253046, -0.25130359,  1.38787176,  1.47154451,
         -1.80649616,  1.09296291,  0.94281138,  0.52836084,
          0.61258103,  0.87656257,  1.47801536,  1.71982086,
          1.28221181,  1.39086472,  1.58209844,  1.08872091,
          0.55212962, -1.81398118,  1.44113674,  1.11191094,
          1.30470835,  0.86390516, -1.67365968, -1.70738939,
         -1.55813196, -0.48498301, -0.75714639,  0.40770893,
         -0.27394734,  2.02323219,  1.2908951 ,  1.70465513,
          1.81148084,  1.39968653, -1.36191442,  1.28024593,
          1.11054795,  0.12662799,  1.40414112,  1.40468999,
          1.45643959,  1.58509336,  1.32283157,  1.30222047,
          1.55138602,  1.91801602, -0.77489784,  2.32098884,
         -1.1942541 , -1

In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     2685.740000
1     3070.312000
2     3399.177000
3     3487.789000
4     3505.720000
5     3226.786714
6     2978.617371
7     2744.168914
8     2703.746000
9     2665.575000
10    2753.564000
11    2993.250000
Name: Rio Grande do Sul - Consumo de Cimento (t), dtype: float64

In [14]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    2993.25
Name: Rio Grande do Sul - Consumo de Cimento (t), dtype: float64

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Rio Grande do Sul - IDH Longevidade,Rio Grande do Sul - IDH,Rio Grande do Sul - IDH Renda,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande do Sul - IDH Educacao,Rio Grande do Sul - Desemprego,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rio Grande Do Sul - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-2.63518,-0.615909,-1.258009,0.317669,-1.204049,-1.609386,-0.865214,1.070122,-0.811701,1.108405,...,-1.482779,-2.260447,-0.920403,-0.082391,0.303877,1.272786,0.19719,-0.13743,-0.327018,0.305695


In [16]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[-2.63518049, -0.61590911, -1.25800882,  0.31766875,
         -1.20404878, -1.60938646, -0.86521381,  1.07012222,
         -0.81170129,  1.10840491,  2.63582309, -0.18995911,
          0.72640489,  2.66833308, -0.032632  ,  0.98802435,
          0.24980494, -0.08955808,  1.85434394,  2.02706568,
         -2.21150703,  1.42938591,  0.85132668,  0.14092386,
          0.30902324,  1.00267904,  2.27970137,  1.11272593,
          0.61007902,  0.89064538,  1.42231438,  1.02650562,
          1.47127593, -1.53201473,  1.73503858,  1.66610613,
          1.0456894 ,  0.55289259, -1.81216593, -2.10342296,
         -2.55377478,  0.41940628, -1.75756445,  0.241023  ,
         -0.98659345,  1.52184916,  1.01398607,  1.49290967,
          1.40764092,  1.87930411, -2.51892072,  2.83356214,
          1.58620924, -0.02524128,  1.54231542,  1.5176119 ,
          1.7355269 ,  1.87941982,  1.54198129,  1.51276712,
          1.83795594,  2.62210693,  1.04993174,  0.96476493,
         -1.48277861, -2

In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    3222.295
Name: Rio Grande do Sul - Consumo de Cimento (t), dtype: float64

In [18]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [19]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [20]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1351278086, 3210884078, 2970683029, 4155816033, 1343252694, 3383706108, 847416203, 1563717878, 4085255625, 1377396199, 627268699, 1113493891, 3962880509, 1040874034, 3672978922, 351459681, 1382127090, 1588519900, 4059803261, 3722965789, 1275399911, 4001437821, 3381804922, 3701048527, 3722926951, 4108912676, 77336370, 3754930428, 2087126845, 4003973690, 2133399674, 3315822938, 3014502176, 2905951689, 3666164998, 3804997508, 407873082, 3355529439, 3288987986, 1372990571, 636688509, 4192177551, 3111124355, 2637502103, 3455568402, 234459695, 2201044515, 970678688, 1503444874, 2898561650, 417450866, 2788882444, 2174623427, 234756644, 3903821014, 3671713795, 993739993, 3725777883, 580445219, 2223341152, 296930935, 4196236144, 4122313318, 3520505696, 4214895014, 1209926204, 1721658767, 2237183133, 1463408579, 2895193585, 556465433, 2918341960, 2859616320, 3940032933, 3565233511, 3328523251, 762160268, 4153741656, 3989536834, 1145107159, 2674813077, 2722016785, 3014743211, 366844919, 29868627

2023-09-09 20:00:07.315087: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-09 20:00:07.335042: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-09 20:00:07.335224: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

loss: 14646.52734375
winner_seed: 1351278086


Step: 1 ___________________________________________
loss: 13061.3974609375
winner_seed: 3210884078


Step: 2 ___________________________________________
loss: 16910.908203125


Step: 3 ___________________________________________
loss: 15344.4560546875


Step: 4 ___________________________________________
loss: 22444.853515625


Step: 5 ___________________________________________
loss: 17085.044921875


Step: 6 ___________________________________________
loss: 12185.5302734375
winner_seed: 847416203


Step: 7 ___________________________________________
loss: 13707.8203125


Step: 8 ___________________________________________
loss: 15239.1435546875


Step: 9 ___________________________________________
loss: 17316.958984375


Step: 10 ___________________________________________
loss: 15482.4404296875


Step: 11 ___________________________________________
loss: 15870.5517578125


Step: 12 ___________________________________________
loss: 15438

In [21]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 3s 3s/step - loss: 9201722.0000
Epoch 2/10000
1/1 [==============================] - 0s 5ms/step - loss: 8800001.0000
Epoch 3/10000
1/1 [==============================] - 0s 5ms/step - loss: 2708046.0000
Epoch 4/10000
1/1 [==============================] - 0s 4ms/step - loss: 1478608.8750
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 3797556.2500
Epoch 6/10000
1/1 [==============================] - 0s 4ms/step - loss: 264606.6250
Epoch 7/10000
1/1 [==============================] - 0s 5ms/step - loss: 529601.0000
Epoch 8/10000
1/1 [==============================] - 0s 5ms/step - loss: 411543.1250
Epoch 9/10000
1/1 [==============================] - 0s 5ms/step - loss: 152442.2656
Epoch 10/10000
1/1 [==============================] - 0s 5ms/step - loss: 367865.5312
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 138995.3594
Epoch 12/10000
1/1 [==============================] - 0s 4m

1/1 [==============================] - 0s 4ms/step - loss: 91820.6328
Epoch 97/10000
1/1 [==============================] - 0s 5ms/step - loss: 34747.2930
Epoch 98/10000
1/1 [==============================] - 0s 5ms/step - loss: 119143.4609
Epoch 99/10000
1/1 [==============================] - 0s 5ms/step - loss: 102773.9297
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 74374.3906
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 71901.4609
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 59839.1094
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 62299.7930
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 71334.0547
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 168346.9531
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 75910.5781
Epoch 107/10000
1/1 [==============================] - 0s 5ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 58912.7539
Epoch 192/10000
1/1 [==============================] - 0s 4ms/step - loss: 62720.9180
Epoch 193/10000
1/1 [==============================] - 0s 5ms/step - loss: 113157.9453
Epoch 194/10000
1/1 [==============================] - 0s 4ms/step - loss: 40078.5781
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 75013.7891
Epoch 196/10000
1/1 [==============================] - 0s 5ms/step - loss: 88062.2891
Epoch 197/10000
1/1 [==============================] - 0s 5ms/step - loss: 69497.8828
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 67013.6797
Epoch 199/10000
1/1 [==============================] - 0s 5ms/step - loss: 59014.4102
Epoch 200/10000
1/1 [==============================] - 0s 5ms/step - loss: 61297.8789
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 56195.4258
Epoch 202/10000
1/1 [==============================] - 0s 5ms/step - 

1/1 [==============================] - 0s 4ms/step - loss: 50135.6367
Epoch 287/10000
1/1 [==============================] - 0s 5ms/step - loss: 118545.1328
Epoch 288/10000
1/1 [==============================] - 0s 4ms/step - loss: 79019.2266
Epoch 289/10000
1/1 [==============================] - 0s 4ms/step - loss: 119761.6250
Epoch 290/10000
1/1 [==============================] - 0s 5ms/step - loss: 48328.0977
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 82216.2891
Epoch 292/10000
1/1 [==============================] - 0s 5ms/step - loss: 45589.7617
Epoch 293/10000
1/1 [==============================] - 0s 5ms/step - loss: 90510.6250
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 50002.5938
Epoch 295/10000
1/1 [==============================] - 0s 4ms/step - loss: 66583.0234
Epoch 296/10000
1/1 [==============================] - 0s 5ms/step - loss: 88354.6797
Epoch 297/10000
1/1 [==============================] - 0s 5ms/step -

1/1 [==============================] - 0s 5ms/step - loss: 69898.5000
Epoch 382/10000
1/1 [==============================] - 0s 5ms/step - loss: 70599.8828
Epoch 383/10000
1/1 [==============================] - 0s 4ms/step - loss: 75355.0234
Epoch 384/10000
1/1 [==============================] - 0s 4ms/step - loss: 34962.7773
Epoch 385/10000
1/1 [==============================] - 0s 5ms/step - loss: 38862.4609
Epoch 386/10000
1/1 [==============================] - 0s 4ms/step - loss: 57262.2812
Epoch 387/10000
1/1 [==============================] - 0s 4ms/step - loss: 90797.6953
Epoch 388/10000
1/1 [==============================] - 0s 5ms/step - loss: 77388.7578
Epoch 389/10000
1/1 [==============================] - 0s 5ms/step - loss: 111373.3828
Epoch 390/10000
1/1 [==============================] - 0s 4ms/step - loss: 100231.6641
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 93744.9453
Epoch 392/10000
1/1 [==============================] - 0s 5ms/step -

...

1/1 [==============================] - 0s 5ms/step - loss: 68870.5547
Epoch 857/10000
1/1 [==============================] - 0s 4ms/step - loss: 76615.9609
Epoch 858/10000
1/1 [==============================] - 0s 4ms/step - loss: 64539.0195
Epoch 859/10000
1/1 [==============================] - 0s 5ms/step - loss: 49470.7461
Epoch 860/10000
1/1 [==============================] - 0s 4ms/step - loss: 81367.4297
Epoch 861/10000
1/1 [==============================] - 0s 4ms/step - loss: 74410.4453
Epoch 862/10000
1/1 [==============================] - 0s 4ms/step - loss: 84713.0781
Epoch 863/10000
1/1 [==============================] - 0s 4ms/step - loss: 62696.7188
Epoch 864/10000
1/1 [==============================] - 0s 4ms/step - loss: 69626.0859
Epoch 865/10000
1/1 [==============================] - 0s 4ms/step - loss: 75280.8281
Epoch 866/10000
1/1 [==============================] - 0s 4ms/step - loss: 61998.7070
Epoch 867/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 63016.0430
Epoch 952/10000
1/1 [==============================] - 0s 4ms/step - loss: 75674.1172
Epoch 953/10000
1/1 [==============================] - 0s 4ms/step - loss: 58535.1836
Epoch 954/10000
1/1 [==============================] - 0s 4ms/step - loss: 87011.8203
Epoch 955/10000
1/1 [==============================] - 0s 4ms/step - loss: 69922.1562
Epoch 956/10000
1/1 [==============================] - 0s 4ms/step - loss: 47580.1914
Epoch 957/10000
1/1 [==============================] - 0s 4ms/step - loss: 69535.7578
Epoch 958/10000
1/1 [==============================] - 0s 4ms/step - loss: 75379.8984
Epoch 959/10000
1/1 [==============================] - 0s 4ms/step - loss: 85710.8438
Epoch 960/10000
1/1 [==============================] - 0s 4ms/step - loss: 77778.1875
Epoch 961/10000
1/1 [==============================] - 0s 5ms/step - loss: 51687.4570
Epoch 962/10000
1/1 [==============================] - 0s 5ms/step - l

1/1 [==============================] - 0s 5ms/step - loss: 30131.0020
Epoch 1046/10000
1/1 [==============================] - 0s 5ms/step - loss: 67130.1328
Epoch 1047/10000
1/1 [==============================] - 0s 5ms/step - loss: 102361.9141
Epoch 1048/10000
1/1 [==============================] - 0s 4ms/step - loss: 71346.3438
Epoch 1049/10000
1/1 [==============================] - 0s 5ms/step - loss: 30647.3438
Epoch 1050/10000
1/1 [==============================] - 0s 5ms/step - loss: 33498.9492
Epoch 1051/10000
1/1 [==============================] - 0s 4ms/step - loss: 61327.8242
Epoch 1052/10000
1/1 [==============================] - 0s 5ms/step - loss: 33192.1797
Epoch 1053/10000
1/1 [==============================] - 0s 4ms/step - loss: 66619.9297
Epoch 1054/10000
1/1 [==============================] - 0s 4ms/step - loss: 93225.6641
Epoch 1055/10000
1/1 [==============================] - 0s 4ms/step - loss: 70784.8203
Epoch 1056/10000
1/1 [==============================] - 0s 

1/1 [==============================] - 0s 4ms/step - loss: 56250.5039
Epoch 1140/10000
1/1 [==============================] - 0s 4ms/step - loss: 25916.0684
Epoch 1141/10000
1/1 [==============================] - 0s 4ms/step - loss: 71771.4766
Epoch 1142/10000
1/1 [==============================] - 0s 5ms/step - loss: 56255.7305
Epoch 1143/10000
1/1 [==============================] - 0s 4ms/step - loss: 88356.3047
Epoch 1144/10000
1/1 [==============================] - 0s 5ms/step - loss: 78455.0547
Epoch 1145/10000
1/1 [==============================] - 0s 5ms/step - loss: 86507.6953
Epoch 1146/10000
1/1 [==============================] - 0s 4ms/step - loss: 100745.2578
Epoch 1147/10000
1/1 [==============================] - 0s 4ms/step - loss: 85383.7891
Epoch 1148/10000
1/1 [==============================] - 0s 4ms/step - loss: 62548.8281
Epoch 1149/10000
1/1 [==============================] - 0s 4ms/step - loss: 39603.0742
Epoch 1150/10000
1/1 [==============================] - 0s 

1/1 [==============================] - 0s 4ms/step - loss: 63376.2852
Epoch 1234/10000
1/1 [==============================] - 0s 4ms/step - loss: 72445.8828
Epoch 1235/10000
1/1 [==============================] - 0s 4ms/step - loss: 90831.0234
Epoch 1236/10000
1/1 [==============================] - 0s 4ms/step - loss: 102937.2812
Epoch 1237/10000
1/1 [==============================] - 0s 4ms/step - loss: 88287.1016
Epoch 1238/10000
1/1 [==============================] - 0s 4ms/step - loss: 95271.7266
Epoch 1239/10000
1/1 [==============================] - 0s 4ms/step - loss: 56065.7070
Epoch 1240/10000
1/1 [==============================] - 0s 4ms/step - loss: 29872.7246
Epoch 1241/10000
1/1 [==============================] - 0s 4ms/step - loss: 59575.7969
Epoch 1242/10000
1/1 [==============================] - 0s 4ms/step - loss: 96183.9766
Epoch 1243/10000
1/1 [==============================] - 0s 4ms/step - loss: 32184.7949
Epoch 1244/10000
1/1 [==============================] - 0s 

In [22]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 587ms/step
[3222.295](test_target) - [2841.6133](prediction) = 380.6817187500001


In [23]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.11813993403769675

In [24]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [25]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Rio Grande do Sul - IDH Longevidade,Rio Grande do Sul - IDH,Rio Grande do Sul - IDH Renda,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande do Sul - IDH Educacao,Rio Grande do Sul - Desemprego,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rio Grande Do Sul - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.342150,-1.214386,1.306502,-0.943366,-1.199724,-1.023611,-0.935294,-1.212564,-0.453717,1.148205,...,1.308949,1.412346,-1.205543,-1.210900,1.384494,-0.257184,1.219074,-0.980702,-1.257698,1.283275
1,0.285101,-0.020460,-0.184435,-0.440757,-0.048595,-0.333278,-0.451004,-0.024009,1.386861,0.140893,...,-0.190804,-0.643250,-0.037540,1.238135,-0.942012,-1.075730,0.011265,1.372772,1.188890,-0.126950
2,1.057048,1.234847,-1.122067,1.384122,1.248319,1.356889,1.386298,1.236573,-0.933144,-1.289098,...,-1.118144,-0.769096,1.243083,-0.027235,-0.442482,1.332914,-1.230338,-0.392070,0.068808,-1.156326


train_target:


,Rio Grande do Sul - Consumo de Cimento (t)
0,2685.740
1,3070.312
2,3399.177


test_input:


...

,Rio Grande do Sul - IDH Longevidade,Rio Grande do Sul - IDH,Rio Grande do Sul - IDH Renda,Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande do Sul - IDH Educacao,Rio Grande do Sul - Desemprego,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rio Grande Do Sul - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-2.63518,-0.615909,-1.258009,0.317669,-1.204049,-1.609386,-0.865214,1.070122,-0.811701,1.108405,...,-1.482779,-2.260447,-0.920403,-0.082391,0.303877,1.272786,0.19719,-0.13743,-0.327018,0.305695


test_target:


,Rio Grande do Sul - Consumo de Cimento (t)
12,3222.295


1/1 [==============================] - 1s 576ms/step
Error: 380.6817187500001




[3338.703125,
 3437.80029296875,
 3428.634521484375,
 3174.91943359375,
 2840.37646484375,
 2871.05810546875,
 2634.53857421875,
 2698.376708984375,
 2728.512451171875,
 2841.61328125]

In [26]:
display(targets)
display(predictions)

[3487.789,
 3505.72,
 3226.7867142857144,
 2978.6173714285715,
 2744.1689142857144,
 2703.746,
 2665.575,
 2753.564,
 2993.25,
 3222.295]

[3338.703125,
 3437.80029296875,
 3428.634521484375,
 3174.91943359375,
 2840.37646484375,
 2871.05810546875,
 2634.53857421875,
 2698.376708984375,
 2728.512451171875,
 2841.61328125]

In [27]:
mae = mean_absolute_error(predictions, targets)
mae

161.03180917968743

In [28]:
porcentage = mae/np.mean(targets)
porcentage

0.0531782591238137